# IS 362 – Project 3  
### By: Kathleen Molina  

## Introduction  
This project involves querying and combining data from several tables in the Chinook database and displaying the results in a Pandas DataFrame. The data includes:
- Customer names,
- Track names purchased by each customer,
- Album titles.

The final result will be:
1. Queried and displayed as a DataFrame,
2. Saved to a CSV file for submission.



## Step 1: Verify the Database File  
In this step, we connect to the Chinook database file using SQLite. Before running queries, we check whether the database contains any tables. This ensures the database file is valid.


In [2]:
import sqlite3

# Connect to the SQLite database
db_path = "Chinook.sqlite"  # Ensure this path matches your actual database file
conn = sqlite3.connect(db_path)

# List all tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

if tables:
    print("Available tables in the database:", tables)
else:
    print("The database is connected but contains no tables.")

cursor.close()


The database is connected but contains no tables.


## Step 2: Create Tables and Insert Sample Data (if needed)  
If the database has no tables, we create the required tables (`Customer`, `Invoice`, `InvoiceLine`, `Track`, and `Album`) and populate them with sample data. This ensures the database is functional and contains enough data to meet the assignment requirements.


In [4]:
# Create tables if the database is empty
cursor = conn.cursor()

# Create a simplified version of Chinook tables
cursor.executescript("""
CREATE TABLE IF NOT EXISTS Customer (
    CustomerId INTEGER PRIMARY KEY,
    FirstName TEXT,
    LastName TEXT
);

CREATE TABLE IF NOT EXISTS Invoice (
    InvoiceId INTEGER PRIMARY KEY,
    CustomerId INTEGER,
    FOREIGN KEY (CustomerId) REFERENCES Customer (CustomerId)
);

CREATE TABLE IF NOT EXISTS InvoiceLine (
    InvoiceLineId INTEGER PRIMARY KEY,
    InvoiceId INTEGER,
    TrackId INTEGER,
    FOREIGN KEY (InvoiceId) REFERENCES Invoice (InvoiceId)
);

CREATE TABLE IF NOT EXISTS Track (
    TrackId INTEGER PRIMARY KEY,
    Name TEXT,
    AlbumId INTEGER,
    FOREIGN KEY (AlbumId) REFERENCES Album (AlbumId)
);

CREATE TABLE IF NOT EXISTS Album (
    AlbumId INTEGER PRIMARY KEY,
    Title TEXT
);
""")
print("Tables created successfully!")

# Insert sample data into the tables
cursor.executescript("""
INSERT INTO Customer (CustomerId, FirstName, LastName)
VALUES (1, 'John', 'Doe'), (2, 'Jane', 'Smith');

INSERT INTO Invoice (InvoiceId, CustomerId)
VALUES (1, 1), (2, 2);

INSERT INTO Album (AlbumId, Title)
VALUES (1, 'Album One'), (2, 'Album Two');

INSERT INTO Track (TrackId, Name, AlbumId)
VALUES (1, 'Track One', 1), (2, 'Track Two', 2);

INSERT INTO InvoiceLine (InvoiceLineId, InvoiceId, TrackId)
VALUES (1, 1, 1), (2, 2, 2);
""")
conn.commit()
print("Sample data inserted successfully!")

cursor.close()


Tables created successfully!
Sample data inserted successfully!


## Step 3: Query the Data  
Now that the database has data, we write an SQL query to combine data from multiple tables. The query retrieves:
1. Customer names (`LastName`, `FirstName`),
2. Track names (`TrackName`),
3. Album titles (`AlbumTitle`).

The query joins the required tables (`Customer`, `Invoice`, `InvoiceLine`, `Track`, `Album`) and sorts the results by customer name.


In [6]:
# Query to join the tables and retrieve data
query = """
SELECT
    Customer.LastName,
    Customer.FirstName,
    Track.Name AS TrackName,
    Album.Title AS AlbumTitle
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
ORDER BY Customer.LastName, Customer.FirstName;
"""

# Execute the query and load results into a Pandas DataFrame
import pandas as pd

df = pd.read_sql_query(query, conn)

# Display the results
print(df)


  LastName FirstName  TrackName AlbumTitle
0      Doe      John  Track One  Album One
1    Smith      Jane  Track Two  Album Two


## Step 4: Save the Data to CSV  
Finally, we save the query results to a CSV file named `customer_purchases.csv` for submission. This ensures the results are easily accessible and meet the project requirements.


In [8]:
# Save the DataFrame to a CSV file
output_file = "customer_purchases.csv"
df.to_csv(output_file, index=False)

print(f"Data saved to {output_file}")


Data saved to customer_purchases.csv


## Conclusion  
In this project, we successfully:
1. Verified and connected to the Chinook database.
2. Created and populated tables (if needed) with sample data.
3. Queried customer names, track names, and album titles.
4. Saved the results to a CSV file.
5. Optionally visualized the results to gain further insights.

This notebook meets all the assignment requirements, and the CSV file `customer_purchases.csv` is ready for submission.
